## Librerias

In [104]:
import pandas as pd
import numpy as np
# pd.options.display.max_rows = 100
pd.options.display.max_columns = 100
# from IPython.core.display import display, HTML
# display(HTML("<style>.container { width:100% !important; }</style>"))

### Funciones

In [83]:
def nas(df):
    zero_val = (df == 0.00).astype(int).sum(axis=0)
    mis_val = df.isnull().sum()
    mis_val_percent = 100 * df.isnull().sum() / len(df)
    mz_table = pd.concat([mis_val, mis_val_percent], axis=1)
    mz_table = mz_table.rename(columns = {0:'Valores_perdidos',1 : 'Valores_Perd_Porc'})
    mz_table['Valores_Perd_Porc'] = mz_table['Valores_Perd_Porc'] # + mz_table['Zero Values'] 
    mz_table['Tipo_dato'] = df.dtypes
    mz_table = mz_table[mz_table.iloc[:,1] != 0].sort_values('Valores_Perd_Porc', ascending=False).round(1)
    print ("El data frame tiene " + str(df.shape[1]) + " columnas y " + str(df.shape[0]) + " filas.\n"      
        "Hay " + str(mz_table.shape[0]) +
          " columnas que tienen valores perdidos.")
    return mz_table


## 1. Descripción del dataset. ¿Por qué es importante y qué pregunta/problema pretende responder?

El dataset escogido es importante porque permite evidenciar la percepción académica, de uso y de calidad de información para los docentes,  del repositorio <a href="https://es.wikipedia.org/wiki/Wikipedia:Portada" target="_blnak">Wikipedia</a>. Los datos provienen de una encuesta enviada a profesores a tiempo parcial y a tiempo completo en dos universidades españolas en 2012-2013: Universitat Oberta de Catalunya (<a href="https://www.uoc.edu/">UOC</a>) y Universitat Pompeu Fabra (<a href="https://www.upf.edu/es/">UPF</a>).


En esta idea, el <b>dataset</b> ofrece la oportunidad de conocer, a través de una encuesta realizada con variables categoricas, discretas y continuas, cual es la percepción de los docentes de educación superior con respecto al aporte de wikipedia, como herramienta que contribuya, con su información, a los estudiantes.

Pregunta problema: <b>¿Cuál es la percepción de los docentes de las universidades UOC y UPF con respecto a wikipedia como herramienta de colaboración a los estudiantes?</b>

## 2. Integración y selección de los datos de interés a analizar.

Inicialmente, se cargaran los datos y se realizará una descripción estadística general; de igual forma, se presentaran los primeros registros con el fin de conocer los valores tomados por variable:  

### Cargue de Datos

In [88]:
data = pd.read_csv('wiki4HE.csv', sep=";")
print("El dataset cuenta con ", data.shape[0], " filas y ", data.shape[1], " variables")
data.head()

El dataset cuenta con  913  filas y  53  variables


,AGE,GENDER,DOMAIN,PhD,YEARSEXP,UNIVERSITY,UOC_POSITION,OTHER_POSITION,OTHERSTATUS,USERWIKI,PU1,PU2,PU3,PEU1,PEU2,PEU3,ENJ1,ENJ2,Qu1,Qu2,Qu3,Qu4,Qu5,Vis1,Vis2,Vis3,Im1,Im2,Im3,SA1,SA2,SA3,Use1,Use2,Use3,Use4,Use5,Pf1,Pf2,Pf3,JR1,JR2,BI1,BI2,Inc1,Inc2,Inc3,Inc4,Exp1,Exp2,Exp3,Exp4,Exp5
0,40,0,2,1,14,1,2,?,?,0,4,4,3,5,5,3,4,4,3,3,2,2,3,3,3,1,2,4,2,4,4,4,3,1,1,2,4,1,1,3,4,1,2,3,5,5,5,5,4,4,4,1,2
1,42,0,5,1,18,1,2,?,?,0,2,3,3,4,4,3,3,4,4,4,3,3,2,3,3,1,1,2,1,4,4,4,2,2,1,1,2,1,1,1,4,3,2,2,4,4,3,4,2,2,4,2,4
2,37,0,4,1,13,1,3,?,?,0,2,2,2,4,4,3,3,3,2,2,2,5,3,2,3,2,2,4,3,4,2,4,1,1,1,1,1,3,3,3,4,3,1,1,5,3,5,5,2,2,2,1,3
3,40,0,4,0,13,1,3,?,?,0,3,3,4,3,3,3,4,3,3,4,3,3,3,3,4,3,2,2,3,5,5,5,3,3,3,3,3,4,4,4,5,4,3,3,3,4,4,3,4,4,3,3,4
4,51,0,6,0,8,1,3,?,?,1,4,3,5,5,4,3,4,4,4,5,4,3,4,4,4,4,3,2,3,5,5,5,3,3,5,5,5,5,5,5,3,3,5,5,5,5,4,4,5,5,5,4,4


### Resumen

In [89]:
data.describe()

,AGE,GENDER,PhD,UNIVERSITY
count,913.000000,913.000000,913.000000,913.000000
mean,42.246440,0.424973,0.464403,1.123768
std,8.058418,0.494610,0.499005,0.329497
min,23.000000,0.000000,0.000000,1.000000
25%,36.000000,0.000000,0.000000,1.000000
50%,42.000000,0.000000,0.000000,1.000000
75%,47.000000,1.000000,1.000000,1.000000
max,69.000000,1.000000,1.000000,2.000000


Como se logra observar en el resumen anterior, de las 53 variables, tan solo 4 (__AGE, GENDER, PhD y University__) han sido tomadas como numéricas. Con esto, se debe realizar un cambio de tipos de variable. También, es de mencionar que este dataset podría ser dividido en 2 datasets: __variables demográficas__ y __variables de aspecto sobre wikipedia__.

#### Tipos de Datos (por defecto)

A continuación se observa como python ha cargado los datos, así como su tipo de variable:

In [90]:
data.columns.to_series().groupby(data.dtypes).groups

{dtype('int64'): Index(['AGE', 'GENDER', 'PhD', 'UNIVERSITY'], dtype='object'),
 dtype('O'): Index(['DOMAIN', 'YEARSEXP', 'UOC_POSITION', 'OTHER_POSITION', 'OTHERSTATUS',
        'USERWIKI', 'PU1', 'PU2', 'PU3', 'PEU1', 'PEU2', 'PEU3', 'ENJ1', 'ENJ2',
        'Qu1', 'Qu2', 'Qu3', 'Qu4', 'Qu5', 'Vis1', 'Vis2', 'Vis3', 'Im1', 'Im2',
        'Im3', 'SA1', 'SA2', 'SA3', 'Use1', 'Use2', 'Use3', 'Use4', 'Use5',
        'Pf1', 'Pf2', 'Pf3', 'JR1', 'JR2', 'BI1', 'BI2', 'Inc1', 'Inc2', 'Inc3',
        'Inc4', 'Exp1', 'Exp2', 'Exp3', 'Exp4', 'Exp5'],
       dtype='object')}

#### Tipos de Datos (modificados)

In [92]:
vars_no_category = ["AGE", "YEARSEXP"]
for col in data.loc[:, ~data.columns.isin(vars_no_category)].columns:
    data[col] = data[col].astype('category')


##### Resumen Tipos de variable

In [93]:
pd.DataFrame(data.dtypes, columns=['Tipo'])


,Tipo
AGE,int64
GENDER,category
DOMAIN,category
PhD,category
YEARSEXP,object
UNIVERSITY,category
UOC_POSITION,category
OTHER_POSITION,category
OTHERSTATUS,category
USERWIKI,category


Ahora bien, se observa que la variable __YEARSEXP__ ha sido cargada como "object" a pesar de que es numerica (según la descripción de la <a href="https://archive.ics.uci.edu/ml/datasets/wiki4HE">página del set oficial</a>). Sin embargo, esta variable cuenta con valores nulos, por lo que sera tratada en un apartado siguiente.

#### Conocimiento de los datos: Variables Demográficas

A continuación se presentan las variables demográficas del dataset:

In [94]:
var = ["AGE", "GENDER", "DOMAIN", "PhD", "YEARSEXP", "UNIVERSITY", "UOC_POSITION", "OTHER","OTHER_POSITION", "USERWIKI"]
descrip = ["Age of Professor",
                "Male or Female",
                "Area of Expertise: Arts & Humanities; Sciences; Health Sciences; Engineering & Architecture; Law & Politics; Social Sciences",
                "Has PhD or Not",
                "Years of university teaching experience",
                "Universitat Oberta de Catalunya (UOC) or Universitat Pompeu Fabra (UPF).",
                "Professor; Associate; Assistant; Lecturer; Instructor; Adjunct", 
                "For part time members, states if main job in another university",
                "People who work as part time in another university and UPF members",
                "Wether a registered Wikipedia user or not"]
pd.DataFrame(descrip, var, columns=['Descripción variable'])

,Descripción variable
AGE,Age of Professor
GENDER,Male or Female
DOMAIN,Area of Expertise: Arts & Humanities; Sciences...
PhD,Has PhD or Not
YEARSEXP,Years of university teaching experience
UNIVERSITY,Universitat Oberta de Catalunya (UOC) or Unive...
UOC_POSITION,Professor; Associate; Assistant; Lecturer; Ins...
OTHER,"For part time members, states if main job in a..."
OTHER_POSITION,People who work as part time in another univer...
USERWIKI,Wether a registered Wikipedia user or not


#### Conocimiento de las variables de aspecto

In [101]:
var = ["Perceived Usefulness", "Perceived Ease of Use", "Perceived Enjoyment", 
            "Quality", "Visibility", "Social Image", "Sharing Attitude", "Use Behaviour",
            "Profile 2.0", "Job relevance","Behavioral intention ", "Incentives","Experience"]
descrip = ["""PU1: The use of Wikipedia makes it easier for students to develop new skills,
           PU2: The use of Wikipedia improves students learning,
PU3: Wikipedia is useful for teaching """,
                """PEU1: Wikipedia is user-friendly,
PEU2: It is easy to find in Wikipedia the information you seek,
PEU3: It is easy to add or edit information in Wikipedia \n""",
                """ENJ1: The use of Wikipedia stimulates curiosity ,
ENJ2: The use of Wikipedia is entertaining """,
                """QU1: Articles in Wikipedia are reliable,
QU2: Articles in Wikipedia are updated ,
QU3: Articles in Wikipedia are comprehensive,
QU4: In my area of expertise, Wikipedia has a lower quality than other educational resources,
QU5: I trust in the editing system of Wikipedia """, 
                """VIS1: Wikipedia improves visibility of students' work,
VIS2: It is easy to have a record of the contributions made in Wikipedia,
VIS3: I cite Wikipedia in my academic papers """,
                """IM1: The use of Wikipedia is well considered among colleagues,
IM2: In academia, sharing open educational resources is appreciated,
IM3: My colleagues use Wikipedia """, 
                """SA1: It is important to share academic content in open platforms,
SA2: It is important to publish research results in other media than academic journals or books,
SA3: It is important that students become familiar with online collaborative environments """, 
                """USE1: I use Wikipedia to develop my teaching materials,
USE2: I use Wikipedia as a platform to develop educational activities with students,
USE3: I recommend my students to use Wikipedia,
USE4: I recommend my colleagues to use Wikipedia,
USE5: I agree my students use Wikipedia in my courses """, 
                """PF1: I contribute to blogs,
PF2: I actively participate in social networks,
PF3: I publish academic content in open platforms """, 
               """JR1: My university promotes the use of open collaborative environments in the Internet,
JR2: My university considers the use of open collaborative environments in the Internet as a teaching merit """,
                """BI1: In the future I will recommend the use of Wikipedia to my colleagues and students,
BI2: In the future I will use Wikipedia in my teaching activity """,
                """INC1: To design educational activities using Wikipedia, it would be helpful: a best practices guide,
INC2: To design educational activities using Wikipedia, it would be helpful: getting instruction from a colleague,
INC3: To design educational activities using Wikipedia, it would be helpful: getting specific training,
INC4: To design educational activities using Wikipedia, it would be helpfull: greater institutional recognition """,
                """EXP1: I consult Wikipedia for issues related to my field of expertise,
EXP2: I consult Wikipedia for other academic related issues,
EXP3: I consult Wikipedia for personal issues,
EXP4: I contribute to Wikipedia (editions, revisions, articles improvement...) ,
EXP5: I use wikis to work with my students"""]
pd.DataFrame(descrip, var, columns=['Descripción variable'])

,Descripción variable
Perceived Usefulness,PU1: The use of Wikipedia makes it easier for ...
Perceived Ease of Use,"PEU1: Wikipedia is user-friendly,\nPEU2: It is..."
Perceived Enjoyment,ENJ1: The use of Wikipedia stimulates curiosit...
Quality,"QU1: Articles in Wikipedia are reliable,\nQU2:..."
Visibility,VIS1: Wikipedia improves visibility of student...
Social Image,IM1: The use of Wikipedia is well considered a...
Sharing Attitude,SA1: It is important to share academic content...
Use Behaviour,USE1: I use Wikipedia to develop my teaching m...
Profile 2.0,"PF1: I contribute to blogs,\nPF2: I actively p..."
Job relevance,JR1: My university promotes the use of open co...


## 3 Limpieza de los datos.

### 3.1 ¿Los datos contienen ceros o elementos vacíos? ¿Cómo gestionarías cada uno de estos casos?

#### Validación de Nulos y valores perdidos

Se ha realizado una función que obtiene los valores nulos y su proporción con respecto al dataset

In [108]:
nas(data)

El data frame tiene 53 columnas y 913 filas.
Hay 0 columnas que tienen valores perdidos.


,Valores_perdidos,Valores_Perd_Porc,Tipo_dato


#### Cantidad de Variables con valores atípicos

En una primera impresión, se resalta que los datos cuentan con un valor atípico ('?'), el cual inicialmente no es capturado por funciones de identificación de nan. Por esto, se realizó una búsqueda con _isin_ para obtener todas aquellas columnas con estos valores:

In [107]:
print(data[data.isin(['?'])].dropna(how='all').count())

AGE                 0
GENDER              0
DOMAIN              2
PhD                 0
YEARSEXP           23
UNIVERSITY          0
UOC_POSITION      113
OTHER_POSITION    261
OTHERSTATUS       540
USERWIKI            4
PU1                 7
PU2                11
PU3                 5
PEU1                4
PEU2               14
PEU3               97
ENJ1                7
ENJ2               17
Qu1                 7
Qu2                10
Qu3                15
Qu4                22
Qu5                29
Vis1               72
Vis2              117
Vis3                8
Im1                22
Im2                20
Im3                57
SA1                11
SA2                12
SA3                11
Use1               14
Use2               17
Use3                9
Use4               23
Use5               15
Pf1                11
Pf2                 6
Pf3                14
JR1                27
JR2                53
BI1                32
BI2                43
Inc1               35
Inc2      

Como se pudó observar en el anterior paso, algunas variables cuentan con valores "?", por lo que a continuación se presenta un conteo por variable:

In [112]:
for col in data.loc[:, ~data.columns.isin(vars_no_category)].columns:
    print(data.groupby([col]).size().reset_index(name='cantidad'))

  GENDER  cantidad
0      0       525
1      1       388
  DOMAIN  cantidad
0      1       183
1      2        56
2      3        73
3      4       137
4      5       101
5      6       361
6      ?         2
  PhD  cantidad
0   0       489
1   1       424
  UNIVERSITY  cantidad
0          1       800
1          2       113
  UOC_POSITION  cantidad
0            1         3
1            2        68
2            3        50
3            4        18
4            5         2
5            6       659
6            ?       113
  OTHER_POSITION  cantidad
0              1       268
1              2       384
2              ?       261
  OTHERSTATUS  cantidad
0           1        19
1           2       130
2           3        24
3           4        36
4           5        16
5           6        41
6           7       107
7           ?       540
  USERWIKI  cantidad
0        0       784
1        1       125
2        ?         4
  PU1  cantidad
0   1        35
1   2       216
2   3       330
3 

### 3.2 Identificación y tratamiento de valores extremos.

Una vez comprendido que los valores perdidos han sido etiquetados con un caracter especial, se procede a reemplazarlo con _nan_ con el fin de hacer uso de la función de valores perdidos y su proporción en el dataset:

In [113]:
data = data.replace('?', np.nan)
nas(data)

El data frame tiene 53 columnas y 913 filas.
Hay 49 columnas que tienen valores perdidos.


,Valores_perdidos,Valores_Perd_Porc,Tipo_dato
OTHERSTATUS,540,59.1,category
OTHER_POSITION,261,28.6,category
Vis2,117,12.8,category
UOC_POSITION,113,12.4,category
PEU3,97,10.6,category
Vis1,72,7.9,category
Im3,57,6.2,category
JR2,53,5.8,category
BI2,43,4.7,category
Inc4,42,4.6,category


Como se puede observar, existen 49 variables con valores perdidos; siendo _OTHERSTATUS_ la variable con mayor datos perdidos, alcanzando un 59.1% de datos perdidos con respecto al dataset.

### Análisis de los datos.

#### 4.1 Selección de los grupos de datos que se quieren analizar/comparar (planificación de los análisis a aplicar)

#### 4.2 Comprobación de la normalidad y homogeneidad de la varianza.

#### 4.3 Aplicación de pruebas estadísticas para comparar los grupos de datos. En función de los datos y el objetivo del estudio, aplicar pruebas de contraste de hipótesis, correlaciones, regresiones, etc. Aplicar al menos tres métodos de análisis diferentes

### 5 Representación de los resultados a partir de tablas y gráficas.

### 6 Resolución del problema. A partir de los resultados obtenidos, ¿cuáles son las conclusiones? ¿Los resultados permiten responder al problema?